CTGAN Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `CTGAN` model, including how to:

-   Create an instance of `CTGAN`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `CTGAN` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is CTGAN?
--------------

The `sdv.tabular.CTGAN` model is based on the GAN-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `CTGAN` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `ctgan` installed on your
system. If you have not done it yet, please install `ctgan` now by
executing the command `pip install sdv` in a terminal.

</div>

In [1]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they where offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `CTGAN` to learn this data and then sample synthetic data
about new students to see how well de model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.CTGAN` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [2]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying CTGANSynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [3]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [4]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17451,F,53.023284,80.349942,Commerce,66.207381,Sci&Tech,False,0,92.442754,Mkt&HR,52.439696,29830.462919,True,2020-06-23,NaT,6.0
1,17469,M,59.816401,95.401121,Commerce,62.548136,Sci&Tech,False,0,94.902181,Mkt&Fin,59.543711,26513.261370,True,NaT,NaT,3.0
2,17368,M,94.988123,80.077772,Science,64.407797,Comm&Mgmt,True,0,58.945849,Mkt&HR,75.918706,28784.473968,False,2020-05-22,NaT,6.0
3,17475,F,64.382464,78.378936,Commerce,67.240249,Comm&Mgmt,False,0,43.886476,Mkt&HR,72.009873,NaN,False,2020-01-07,2020-03-16,3.0
4,17338,M,85.364890,59.563715,Science,70.771859,Comm&Mgmt,True,0,93.544320,Mkt&Fin,61.524893,NaN,False,2020-02-09,NaT,NaN


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [5]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `CTGAN.load` method, and then you are ready to sample new data
from the loaded instance:

In [6]:
loaded = CTGAN.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `ctgan` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [7]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [8]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
62,17443,M,88.721661,65.453379,Arts,73.523980,Comm&Mgmt,False,0,88.003668,Mkt&Fin,56.539360,25130.436842,False,NaT,NaT,12.0
152,17443,M,94.078697,74.069296,Commerce,76.661577,Comm&Mgmt,False,0,69.040908,Mkt&Fin,52.719936,28299.955878,True,NaT,NaT,12.0
163,17443,M,67.728886,56.330433,Commerce,51.527745,Comm&Mgmt,True,0,51.181985,Mkt&HR,66.779506,NaN,False,NaT,2020-08-31,NaN
196,17443,F,64.148263,77.340672,Commerce,67.582524,Others,False,0,51.106249,Mkt&HR,59.628052,NaN,True,2019-12-11,NaT,6.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [9]:
model = CTGAN(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,50.780957,58.223332,Science,71.312818,Sci&Tech,False,0,77.688814,Mkt&Fin,55.776461,31827.867296,False,2019-12-23,NaT,3.0
1,1,F,74.582301,72.643448,Arts,63.582570,Comm&Mgmt,False,0,90.977080,Mkt&Fin,60.245338,42205.226861,False,2020-03-19,2020-07-15,NaN
2,2,M,48.939718,80.952561,Commerce,59.245101,Sci&Tech,False,0,68.763742,Mkt&Fin,53.347388,31124.559816,True,NaT,2020-08-04,6.0
3,3,M,76.271200,35.515856,Science,47.396723,Sci&Tech,False,0,74.733539,Mkt&HR,63.243934,NaN,True,NaT,2020-07-16,6.0
4,4,M,53.275420,62.719193,Commerce,56.439593,Comm&Mgmt,False,0,62.335988,Mkt&Fin,62.479833,32516.022617,True,2020-03-26,2020-05-11,3.0


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [10]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [11]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [12]:
model = CTGAN(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"50704 Branch Pine Apt. 243\nJesseville, PA 26331",F,61.569642,53.173203,Science,69.728100,Comm&Mgmt,False,0,61.712515,Mkt&HR,71.477420,20485.395347,True,2020-06-04,NaT,3.0
1,1,"3702 Bradley Island\nNorth Victor, FL 12268",F,63.446271,68.721759,Science,82.300095,Sci&Tech,True,0,48.210307,Mkt&HR,70.724087,27829.977238,True,2019-12-23,NaT,NaN
2,2,"835 Patel Trace Apt. 213\nWoodtown, NY 30903",M,80.864938,79.713387,Science,50.368845,Comm&Mgmt,True,0,49.208499,Mkt&Fin,65.590205,28749.567072,True,2020-01-13,2020-04-17,12.0
3,3,"75756 Anthony Prairie\nNorth Charlene, CA 60546",M,59.435521,63.124106,Science,62.824996,Sci&Tech,True,0,96.570265,Mkt&Fin,59.317351,28949.991884,False,2019-12-31,NaT,3.0
4,4,"70304 Baker Turnpike\nEricborough, MS 15086",M,70.675986,84.705093,Commerce,52.440202,Comm&Mgmt,False,0,51.673012,Mkt&Fin,62.179705,34841.792490,False,2020-05-24,2020-05-28,NaN


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [13]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [14]:
model = CTGAN(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [15]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,USNV Harris\nFPO AE 40326,M,75.535040,45.146135,Arts,63.396996,Comm&Mgmt,True,0,94.258984,Mkt&HR,50.608845,NaN,True,2020-06-14,2020-08-30,NaN
1,1,"74712 Rich Point\nJohnsontown, NH 14332",M,77.228626,68.227249,Science,59.479151,Comm&Mgmt,False,1,96.626266,Mkt&Fin,51.756924,27321.561556,False,2020-01-19,2020-08-09,3.0
2,2,"38042 Rios Camp Apt. 628\nCarlyhaven, HI 23946",F,71.350786,67.089178,Science,52.932296,Sci&Tech,True,0,101.652924,Mkt&HR,49.093051,28916.429091,True,2019-12-29,NaT,NaN
3,3,"510 Amber Terrace\nWest Daniel, NY 14568",F,73.569160,60.484782,Science,66.736718,Sci&Tech,True,0,76.820654,Mkt&Fin,56.682990,27350.961612,True,2020-06-14,2020-08-12,3.0
4,4,"82223 Peggy Road\nNorth Elizabethburgh, SD 24587",F,67.542573,28.796328,Science,69.457403,Comm&Mgmt,False,0,65.925041,Mkt&HR,46.619885,29837.544468,True,2020-01-07,2020-09-10,6.0


Which means that none of the original addresses can be found in the
sampled data:

In [16]:
data_pii.address.isin(new_data_pii.address).sum()

0

Advanced Usage
--------------

Now that we have discovered the basics, let\'s go over a few more
advanced usage examples and see the different arguments that we can pass
to our `CTGAN` Model in order to customize it to our needs.

### How to modify the CTGAN Hyperparameters?

A part from the common Tabular Model arguments, `CTGAN` has a number of
additional hyperparameters that control its learning behavior and can
impact on the performance of the model, both in terms of quality of the
generated data and computational time.

-   `epochs` and `batch_size`: these arguments control the number of
    iterations that the model will perform to optimize its parameters,
    as well as the number of samples used in each step. Its default
    values are `300` and `500` respectively, and `batch_size` needs to
    always be a value which is multiple of `10`. These hyperparameters
    have a very direct effect in time the training process lasts but
    also on the performance of the data, so for new datasets, you might
    want to start by setting a low value on both of them to see how long
    the training process takes on your data and later on increase the
    number to acceptable values in order to improve the performance.
-   `log_frequency`: Whether to use log frequency of categorical levels
    in conditional sampling. It defaults to `True`. This argument affects
    how the model processes the frequencies of the categorical values that
    are used to condition the rest of the values. In some cases, changing
    it to `False` could lead to better performance.
-   `embedding_dim` (int): Size of the random sample passed to the
    Generator. Defaults to 128.
-   `generator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Residuals. A Resiudal Layer will be created for each
    one of the values provided. Defaults to (256, 256).
-   `discriminator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Discriminator Layers. A Linear Layer will be created
    for each one of the values provided. Defaults to (256, 256).
-   `generator_lr` (float): Learning rate for the generator. Defaults to 2e-4.
-   `generator_decay` (float): Generator weight decay for the Adam Optimizer.
    Defaults to 1e-6.
-   `discriminator_lr` (float): Learning rate for the discriminator.
    Defaults to 2e-4.
-   `discriminator_decay` (float): Discriminator weight decay for the Adam
    Optimizer. Defaults to 1e-6.
-   `discriminator_steps` (int): Number of discriminator updates to do for
    each generator update. From the WGAN paper: https://arxiv.org/abs/1701.07875.
    WGAN paper default is 5. Default used is 1 to match original CTGAN
    implementation.
-   `verbose`: Whether to print fit progress on stdout. Defaults to
    `False`.

<div class="alert alert-warning">

**Warning**

Notice that the value that you set on the `batch_size` argument must
always be a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing
the number of epochs, reducing the `batch_size`, adding one additional
layer to the models involved and using a smaller wright decay.

Before we start, we will evaluate the quality of the previously
generated data using the `sdv.evaluation.evaluate` function

In [17]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sdmetrics/column_pairs/statistical/kl_divergence.py:51: SettingWithCopyWarning: 
A value is try

0.3061542000147366

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [18]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    generator_dim=(256, 256, 256),
    discriminator_dim=(256, 256, 256)
)

And fit to our data.

model.fit(data)

Finally, we are ready to generate new data and evaluate the results.

In [51]:
new_data = model.sample(len(data))
evaluate(new_data, data)

0.3013820502960063

### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the CTGAN model. This means we can sample only values that satisfy a certain condition, such as `gender = M`. The conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case it is passed as a dataframe, the model will generate one sample for each row of the `conditions` dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples with `gender = M` and three samples with `gender = F`, we can do the following:

In [52]:
#it currently doesn't break if passed this:  pd.df{"gender": ["M"]*5 + ["F"]*5 + ["a"]}
#it does break if passed {"gender": 'M'} without specifying number of samples. Error unreadable

import pandas as pd
import warnings

warnings.filterwarnings("ignore")
conditions = pd.DataFrame({"gender": ['M']*3 + ['F']*3})
model = CTGAN(epochs=1)
model.fit(data)
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17391,M,63.637064,54.703154,Commerce,88.560102,Comm&Mgmt,False,0,78.812228,Mkt&Fin,58.648434,21826.568466,True,2020-08-21,2020-07-16,12.0
1,17470,M,41.603392,70.579425,Science,75.496333,Sci&Tech,False,0,60.026174,Mkt&HR,51.774750,26424.966708,True,NaT,2020-08-19,6.0
2,17291,M,57.559131,68.918012,Arts,87.990750,Sci&Tech,False,0,90.678263,Mkt&Fin,65.033297,NaN,True,NaT,2020-08-26,3.0
3,17545,F,79.905076,66.165432,Commerce,88.823809,Sci&Tech,True,0,73.359115,Mkt&HR,54.363478,30120.023137,False,2020-03-31,NaT,3.0
4,17286,F,71.261168,67.387472,Arts,75.350856,Comm&Mgmt,True,0,71.327367,Mkt&Fin,70.459199,26277.862302,True,2020-01-30,NaT,6.0
5,17446,F,63.577646,61.383335,Science,63.971443,Sci&Tech,False,0,96.052951,Mkt&HR,76.517942,NaN,False,NaT,NaT,6.0


`conditions` can also be passed as a dictionary. If a dictionary of lists is passed, the behaviour will be the same as for the dataframe. If only a single value is passed however, the method will also expect the number of rows to be sampled as a parameter. For example, if we wanted to generate a thousand samples for `gender = M`, we can run:

In [53]:
conditions = {"gender": 'M'}
model.sample(1000, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17260,M,57.727405,65.930712,Arts,81.645656,Sci&Tech,True,2,68.791072,Mkt&HR,54.394777,36044.223096,True,2020-09-26,2020-08-20,12.0
1,17424,M,72.551097,45.140396,Commerce,78.170638,Others,True,1,47.313669,Mkt&Fin,61.403151,NaN,False,2020-06-26,NaT,3.0
2,17366,M,87.335169,68.205872,Commerce,64.517288,Comm&Mgmt,True,0,80.598218,Mkt&Fin,57.538671,NaN,True,NaT,2020-05-26,12.0
3,17215,M,63.435385,62.561037,Arts,74.081060,Sci&Tech,True,0,94.023828,Mkt&HR,60.443988,NaN,False,2020-03-14,2020-12-01,3.0
4,17313,M,60.568408,72.165665,Arts,73.234326,Comm&Mgmt,True,0,85.323481,Mkt&Fin,60.434305,34646.268254,True,2020-03-03,NaT,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,17348,M,63.079763,81.477006,Arts,81.207854,Comm&Mgmt,False,0,102.230590,Mkt&HR,61.672914,26215.875152,True,2020-02-22,NaT,12.0
996,17479,M,57.979039,78.506342,Arts,85.760847,Others,True,0,55.242735,Mkt&Fin,59.264136,46535.912587,False,NaT,2020-11-16,3.0
997,17283,M,61.616126,84.343792,Arts,81.639209,Sci&Tech,False,0,100.265919,Mkt&Fin,69.184199,24693.958314,False,2020-07-08,NaT,6.0
998,17463,M,57.875824,55.425754,Science,82.771185,Others,False,1,72.682490,Mkt&HR,57.048640,NaN,False,2020-02-25,NaT,6.0


Currently CTGAN only supports conditioning on categorical columns, and only one column can be conditioned at a time.

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.